# Aufbereitung Abstimmungsdaten Grossrat
Im Berner Grossrat wird elektronisch abgestimmt. Allerdings wechselte die Anlage und damit das Datenformat in der laufenden Legislatur mehrfach. So kam bei der ersten Session, die wegen Corona im Messeareal BEA Expo durchgeführt wurde, eine andere Anlage zum Einsatz als zuvor im Rathaus. Und bei der zweiten Expo-Session entstand wiederum ein anders gegliederter Datensatz. In den Daten, die von [Roland Schneeberger](roland.schneeberger@be.ch) von der Staatskanzlei exportiert worden sind, sind teilweise zudem Testabstimmungen enthalten. Diese müssen entfernt werden. Weitere Herausforderung: Unter- und Variantenabstimmungen haben das selbe Gewicht wie Schlussabstimmungen. 

In diesem Notebook werden die Daten der verschiedenen Abstimmungsanlagen aufbereitet und harmonisiert. 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Rathaus" data-toc-modified-id="Rathaus-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Rathaus</a></span></li><li><span><a href="#BEA-Session-1" data-toc-modified-id="BEA-Session-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>BEA-Session 1</a></span></li><li><span><a href="#BEA-Session-2" data-toc-modified-id="BEA-Session-2-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>BEA-Session 2</a></span></li><li><span><a href="#Bea-Session-3-(Festhalle)" data-toc-modified-id="Bea-Session-3-(Festhalle)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Bea-Session 3 (Festhalle)</a></span></li><li><span><a href="#Harmonisieren-und-zusammenfügen-der-Daten" data-toc-modified-id="Harmonisieren-und-zusammenfügen-der-Daten-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Harmonisieren und zusammenfügen der Daten</a></span></li></ul></div>

In [87]:
# Die nötigen Bilbiotheken werden importiert. 
import pandas as pd
from os import listdir
from tqdm import tqdm
import numpy as np
from fuzzywuzzy import process
from requests import get
from bs4 import BeautifulSoup
from time import sleep

In [88]:
# Welche Datei zu welcher Session gehört, wird manuell erfasst. 
format_rathaus = listdir('daten/rathaus/')
format_bea_v1 = listdir('daten/bea1/')
format_bea_v2 = listdir('daten/bea2/')
format_bea_v3 = listdir('daten/bea3/')

# Sperrdateien, die beim Öffnen eines Datensatzes in Office-Programmen entstehen, werden ausgeschlossen.
format_rathaus = [x for x in format_rathaus if 'lock' not in x]
format_bea_v1 = [x for x in format_bea_v1 if 'lock' not in x]
format_bea_v2 = [x for x in format_bea_v2 if 'lock' not in x]
format_bea_v3 = [x for x in format_bea_v3 if 'lock' not in x]

In [164]:
# Eine Liste mit den Parlamentariern wird eingelesen.  
df_grossrat = pd.read_excel('https://www.gr.be.ch/gr/de/index/mitglieder/mitglieder/liste_a-z.assetref/dam/documents/GR/Mitglieder/de/Adressliste--Liste-des-adresses.xlsx')
df_grossrat = df_grossrat.dropna(how='all')

# Eine Korrektur.
i = df_grossrat[df_grossrat['Name'].str.contains('Roulet')].index
df_grossrat.at[i[0], 'Name'] = 'Roulet Romy'
df_grossrat.at[i[0], 'Vornamen'] = 'Sandra'

i = df_grossrat[(df_grossrat['Name'].str.contains('Bauer')) & (df_grossrat['Vornamen'] == 'Tanja')].index
df_grossrat.at[i[0], 'PLZOrt'] = '3084 Wabern'

df_grossrat['name-vorname'] = df_grossrat['Name'] + ' ' + df_grossrat['Vornamen']
df_grossrat['ort'] = df_grossrat['PLZOrt'].apply(lambda x: x.split(' ')[1].strip() if len(x.split(' ')) > 1 else x)

def männleinweiblein(briefanrede): 
    if 'Herr' in briefanrede: 
        return 'm'
    elif 'Monsieur' in briefanrede:
        return 'm'
    elif 'Frau' in briefanrede: 
        return 'w'
    elif 'Madame' in briefanrede:
        return 'w'
    
df_grossrat['geschlecht'] = df_grossrat['Briefanrede'].apply(männleinweiblein)

# Die ehemaligen Parlamentsmitglieder werden nachgetragen. 
ehemalige = [['Luca', 'Alberucci', 'glp', 'Ostermundigen', 'd', 'Mittelland-Nord', 'm'], 
             ['Christian', 'Bachmann', 'SP', 'Nidau', 'd', 'Biel-Bienne - Seeland', 'm'],
             ['Peter', 'Sommer', 'FDP', 'Wynigen', 'd', 'Emmental', 'm'],
             ['Madeleine', 'Graf-Rudolf', 'Grüne', 'Belp', 'd', 'Mittelland-Süd', 'w'],
             ['Marianne', 'Teuscher-Abts', 'FDP', 'Roggwil', 'd', 'Oberaargau', 'w'],
             ['Marianne', 'Burkhard', 'SP', 'Roggwil', 'd', 'Oberaargau', 'w'],
             ['Jakob', 'Etter', 'BDP', 'Treiten', 'd', 'Biel-Bienne - Seeland', 'm'],
             ['Christine', 'Schnegg', 'EVP', 'Lyss', 'd', 'Biel-Bienne - Seeland', 'w'],
             ['Peter', 'Moser', 'FDP', 'Biel/Bienne', 'd', 'Biel-Bienne - Seeland', 'm'],
             ['Stefan', 'Hofer', 'SVP', 'Bern', 'd', 'Bern', 'm'],
             ['Roland', 'Näf', 'SP', 'Muri bei Bern', 'd', 'Mittelland-Nord', 'm'],
             ['Ulrich', 'Stähli', 'BDP', 'Gasel', 'd', 'Mittelland-Süd', 'm'],
             ['Ulrich', 'Frutiger', 'BDP', 'Oberhofen', 'd', 'Thun', 'm'],
             ['Erich', 'Hess', 'SVP', 'Bern', 'd', 'Bern', 'm'],
             ['Vania', 'Kohli', 'BDP', 'Bern', 'd', 'Bern', 'w'],
             ['Samuel', 'Leuenberger', 'BDP', 'Trubschachen', 'd', 'Emmental', 'm'],
             ['Martin', 'Aeschlimann', 'EVP', 'Burgdorf', 'd', 'Emmental', 'm'],
             ['Monika', 'Gygax-Böninger', 'BDP', 'Obersteckholz', 'd', 'Oberaargau', 'w'],
             ['Ruedi', 'Löffel-Wenger', 'EVP', 'Münchenbuchsee', 'd', 'Mittelland-Nord','m'],
             ['Marc', 'Jost', 'EVP', 'Thun', 'd', 'Thun', 'm'],
             ['Jürg', 'Iseli', 'SVP', 'Zwieselberg', 'd', 'Thun', 'm'],
             ['Anita', 'Luginbühl-Bachmann', 'BDP', 'Krattigen', 'd', 'Oberland', 'w'],
             ['Lars', 'Guggisberg', 'SVP', 'Kirchlindach', 'd', 'Mittelland-Nord', 'm'],
             ['Lea', 'Kusano', 'SP', 'Bern', 'd', 'Bern', 'w'],
             ['Tamara', 'Funiciello', 'SP', 'Bern', 'd', 'Bern', 'w'], 
             ['Kilian', 'Baumann', 'Grüne', 'Suberg', 'd', 'Biel-Bienne - Seeland', 'm'],
             ['Daniel', 'Trüssel', 'Grünliberale', 'Trimstein', 'd', 'Mittelland-Süd', 'm'], 
             ['Martin', 'Boss', 'Grüne', 'Saxeten', 'd', 'Oberland', 'm'],
             ['Adrian', 'Wüthrich', 'SP', 'Huttwil', 'd', 'Oberaargau', 'm'],
             ['Sabina', 'Geissbühler-Strupler', 'SVP', 'Herrenschwanden', 'd', 'Mittelland-Nord', 'w'],
             ['Christoph', 'Grimm', 'glp', 'Burgdorf', 'd', 'Emmental', 'm']]

df_ehemalige = pd.DataFrame(ehemalige)
df_ehemalige.columns = ['vorname', 'nachname', 'partei', 'wohnort', 'sprache', 'wahlkreis', 'geschlecht']

df_grossrat = df_grossrat[['Name', 'Vornamen', 'Partei', 'ort', 'Sprache', 'Wahlkreis', 'geschlecht']]
df_grossrat.columns = ['nachname', 'vorname', 'partei', 'wohnort', 'sprache', 'wahlkreis', 'geschlecht']

df_grossrat = df_grossrat.append(df_ehemalige)
df_grossrat = df_grossrat[~df_grossrat['nachname'].isnull()]
df_grossrat.head()

,nachname,vorname,partei,wohnort,sprache,wahlkreis,geschlecht
0,Abplanalp,Ueli,SVP,Brienzwiler,d,Oberland,m
1,Aebi,Markus,SVP,Hellsau,d,Emmental,m
2,Aebischer,Verena,SVP,Guggisberg,d,Mittelland-Süd,w
3,Alberucci,Luca,glp,Ostermundigen,d,Mittelland-Nord,m
4,Ammann,Christa,AL,Bern,d,Bern,w


In [165]:
# Die Namen der Parteien sind unterschiedlich erfasst. Sie werden harmonisiert. 
parteien_dict = {'SVP / UDC': 'SVP', 'SP-JUSO-PSA / PS-JS-PSA': 'SP', 'FDP / PLR': 'FDP',
                 'Grüne / Les Verts': 'Grüne', 'BDP / PBD': 'Die Mitte', 'glp / pvl': 'glp', 
                 'EVP / PEV': 'EVP', 'EDU / UDF': 'EDU', 'CVP / PDC': 'Die Mitte', 'BDP': 'Die Mitte',
                 'PLR': 'FDP', 'PSA': 'SP', 'CVP': 'Die Mitte', 'PS': 'SP', 'les Verts': 'Grüne',
                 'UDC': 'SVP', 'PEV': 'EVP', 'PDC': 'Die Mitte', 'SVP/UDC': 'SVP', 'FDP/PLR': 'FDP',
                 'Grüne/les Verts': 'Grüne', 'glp/plv': 'glp', 'EVP/PEV': 'EVP', 'EDU/UDF': 'EDU',
                 'Grünliberale': 'glp', 'Les Verts': 'Grüne', 'AL': 'Grüne'}

for i, r in df_grossrat.iterrows():
    if r['partei'] in parteien_dict.keys():
        df_grossrat.at[i, 'partei'] = parteien_dict[r['partei']]

In [92]:
# Die Geburtstagsinfos werden abgerufen.

# Eine Korrektur: Ulrich Frutiger heisst in der Personendatenbank Ueli Frutiger. 
for i, r in df_grossrat[df_grossrat['nachname'] == 'Frutiger'].iterrows():
    df_grossrat.at[i, 'vorname'] = 'Ueli'
    
df_grossrat['nachname-vorname'] = df_grossrat['nachname'] + ' ' + df_grossrat['vorname']

geburi_dict = dict()

for index, row in tqdm(df_grossrat.iterrows(), total=len(df_grossrat)):
    try:
        r = get('https://www.gr.be.ch/gr/de/index/mitglieder/mitglieder/suche.html?query=' + row['nachname'] + '+' + row['vorname'] + '&member-type=all&type=all&gender=all&city=all&city_text=&constituency_text=&party=all&time=all&date_from=&date_to=&submit=Suchen')
        s = BeautifulSoup(r.content, 'lxml')
        l = s.find_all('a', href=True)
        l = [x for x in l if 'suche/mitglied.mid' in x['href']]
        l = 'https://www.gr.be.ch/' + l[0]['href']

        # Die Personenseite wird aufgerufen. 
        r = get(l)
        s = BeautifulSoup(r.content, 'lxml')
        geburtsdatum = s.find('table', {'summary': 'mitglieddetail_personaldata_aural_summary'}).find('td').text.strip()

        eintritt = s.find_all('td', {'headers': 'th3_top_1F'})[-1].text.strip()
        austritt = s.find_all('td', {'headers': 'th3_top_2F'})[-1].text.strip()
        
        bild = 'https://www.gr.be.ch/gr' + s.find('div', {'class': 'img-small-portrait'}).find('img')['src']

        geburi_dict[row['nachname-vorname']] = [geburtsdatum, eintritt, austritt, bild]
        sleep(3)
    except: 
        pass
df_temp = pd.DataFrame.from_dict(geburi_dict, orient='index')
df_temp.columns = ['geburtstag', 'eintritt', 'austritt', 'bild']

df_grossrat = pd.merge(df_grossrat, df_temp, left_on='nachname-vorname', right_index=True, how='left')
del df_grossrat['nachname-vorname']

100%|██████████| 191/191 [11:45<00:00,  3.69s/it]


In [166]:
df_grossrat['nachname-vorname'] = df_grossrat['nachname'] + ' ' + df_grossrat['vorname']

df_temp = pd.DataFrame.from_dict(geburi_dict, orient='index')
df_temp.columns = ['geburtstag', 'eintritt', 'austritt', 'bild']

df_grossrat = pd.merge(df_grossrat, df_temp, left_on='nachname-vorname', right_index=True, how='left')
del df_grossrat['nachname-vorname']

In [167]:
# Zwei manuelle Ergänzungen. 
df_grossrat.reset_index(inplace=True, drop=True)

# Lydia Baumgartner
i = df_grossrat[(df_grossrat['nachname'] == 'Baumgartner') & (df_grossrat['vorname'] == 'Lydia')].index[0]
df_grossrat.at[i, 'geburtstag'] = '28.01.1966'
df_grossrat.at[i, 'eintritt'] = '01.06.2018'
df_grossrat.at[i, 'austritt'] = ''
df_grossrat.at[i, 'bild'] = 'https://www.gr.be.ch/etc/designs/gr/media.cdwsbinary.acq/e0201a268e6949ce9aba69bcd776136a-1664/1/Portrait/Baumgartner_Lydia.jpg'

# Ulrich Frutiger
i = df_grossrat[(df_grossrat['nachname'] == 'Frutiger') & (df_grossrat['vorname'] == 'Ulrich')].index[0]
df_grossrat.at[i, 'geburtstag'] = '12.06.1958'
df_grossrat.at[i, 'eintritt'] = '01.06.2010'
df_grossrat.at[i, 'austritt'] = '30.06.2020'
df_grossrat.at[i, 'bild'] = 'https://www.gr.be.ch/etc/designs/gr/media.cdwsbinary.acq/189eaea16e594938a0d928e80506d27a-1664/23/Original/Frutiger_Ueli.jpg'
df_grossrat.at[i, 'vorname'] = 'Ueli'

In [168]:
# Abspeichern des Datensatzes zu den Parlamentariern. 
df_grossrat.to_csv('daten/grossrat.csv')

In [169]:
# Neu einlesen der Daten. 
df_grossrat = pd.read_csv('daten/grossrat.csv', index_col='Unnamed: 0')

## Rathaus
Bis im Frühling 2020 fanden die Sessionen im Rathaus statt. Die fix installierte Abstimmungsanlage produziert CSV. In den Daten sind allerdings viele Testabstimmungen enthalten. Roland Schneeberger hat diese ausgefiltert. Und er hat die Daten ins Excel-Format konvertiert. Diese Daten werden nachfolgend eingelesen. 

In [5]:
# Ein leerer Dataframe wird erstellt. 
cols = ['nachname', 'wohnort', 'partei', 'stimme']
df_rathaus = pd.DataFrame(columns=cols)

for dateiname in tqdm(format_rathaus):

    # Die Namen der Tabs der betreffenden Datei werden ausgelesen. 
    register = pd.ExcelFile('daten/rathaus/' + dateiname).sheet_names

    for r in register:
        # Die Abstimmungsdaten werden eingelesen. 
        df_roh = pd.read_excel('daten/rathaus/' + dateiname, sheet_name=r, skiprows=1)
        df_roh.columns = cols

        # Die Metainformationen zu den Abstimmungen werden ausgelesen (Datum, Resultat sowie Position in der Datei).
        anfang = df_roh[df_roh['nachname'] == 'Name/Nom'].index
        ende = df_roh[df_roh['nachname'] == '####################'].index

        temp_list = list()
        for i in range(len(ende)):
            if i % 2 == 0:
                temp_list.append(ende[i])
        ende = temp_list

        datetime = df_roh[df_roh['nachname'] == 'Datum/Date']['wohnort'].tolist()
        ja = df_roh[df_roh['nachname'] == 'Ja/Oui']['wohnort'].tolist()
        nein = df_roh[df_roh['nachname'] == 'Nein/Non']['wohnort'].tolist()
        enthalten = df_roh[df_roh['nachname'] == 'Enthalten/Abstentions']['wohnort'].tolist()
        art = df_roh[df_roh['nachname'] == 'Art/Sorte']['wohnort'].tolist()
        mandant = df_roh[df_roh['nachname'] == 'Mandant/Mandant']['wohnort'].tolist()
        nummer = df_roh[df_roh['nachname'] == 'Nummer/Numéro']['wohnort'].tolist()
        traktandum = df_roh[df_roh['nachname'] == 'Traktandum/Affair']['wohnort'].tolist()
        traktandum_txt = df_roh[df_roh['nachname'] == 'Traktandum Beschreibung/Affair description']['wohnort'].tolist()

        tab_meta = list(zip(anfang, ende, datetime, ja, nein, enthalten, art, mandant, nummer, traktandum, traktandum_txt))

        # Der Datensatz wird zusammengestellt. 
        for i in range(len(tab_meta)): 
            df_temp = df_roh.loc[tab_meta[i][0] + 1: tab_meta[i][1] - 1].copy()
            if type(tab_meta[i][2]) == float:
                df_temp['datetime'] = pd.to_datetime(r, dayfirst=True)
            else:
                df_temp['datetime'] = tab_meta[i][2]
            df_temp['res_ja'] = tab_meta[i][3] 
            df_temp['res_nein'] = tab_meta[i][4]   
            df_temp['res_enthaltung'] = tab_meta[i][5]
            df_temp['art'] = tab_meta[i][6]
            df_temp['mandant'] = tab_meta[i][7]
            df_temp['nummer'] = tab_meta[i][8]
            
            # Einzelne Abstimmungen tragen die selbe Nummer. Sie werden unterscheidbar gemacht. 
            df_temp['traktandum'] = tab_meta[i][9] + '_' + str(i)

            df_temp['traktandum_txt'] = tab_meta[i][10]
            df_temp['dateiname'] = dateiname
            df_temp['register'] = r
            df_rathaus = df_rathaus.append(df_temp)

# Stimmen, die unter dem Namen "Delegates" abgegeben wurden, werden ausgefiltert. 
df_rathaus = df_rathaus[~df_rathaus['nachname'].str.contains('Delegate')]

# In einigen Namen gibts Sonderzeichen, die das nachträgliche Matching mit dem Personendatensatz verhindern. 
df_rathaus['nachname'] = df_rathaus['nachname'].str.replace('\xa0', '')

df_rathaus['anlage'] = 'rathaus'
df_rathaus.to_csv('daten/daten_rathaus.csv')
df_rathaus.head()

100%|██████████| 8/8 [01:16<00:00,  9.56s/it]


,nachname,wohnort,partei,stimme,datetime,res_ja,res_nein,res_enthaltung,art,mandant,nummer,traktandum,traktandum_txt,dateiname,register,anlage
13,Iseli,Zwieselberg,SVP / UDC,NaN,2018-06-04 15:47:00,137.0,1.0,1.0,Abstimmung mit Namensliste / Procès-verbal de ...,Grossrat / le grand conseil,11.0,#2017.RRGR.459_0,Eröffnung der Sitzung durch das älteste der an...,Sommersession 2018.xlsx,4.04.2018,rathaus
14,Aebi,Hellsau,SVP / UDC,Ja / Oui,2018-06-04 15:47:00,137.0,1.0,1.0,Abstimmung mit Namensliste / Procès-verbal de ...,Grossrat / le grand conseil,11.0,#2017.RRGR.459_0,Eröffnung der Sitzung durch das älteste der an...,Sommersession 2018.xlsx,4.04.2018,rathaus
15,Aebischer,Riffenmatt,SVP / UDC,Ja / Oui,2018-06-04 15:47:00,137.0,1.0,1.0,Abstimmung mit Namensliste / Procès-verbal de ...,Grossrat / le grand conseil,11.0,#2017.RRGR.459_0,Eröffnung der Sitzung durch das älteste der an...,Sommersession 2018.xlsx,4.04.2018,rathaus
16,Amstutz,Schwanden-Sigriswil,SVP / UDC,Ja / Oui,2018-06-04 15:47:00,137.0,1.0,1.0,Abstimmung mit Namensliste / Procès-verbal de ...,Grossrat / le grand conseil,11.0,#2017.RRGR.459_0,Eröffnung der Sitzung durch das älteste der an...,Sommersession 2018.xlsx,4.04.2018,rathaus
17,Augstburger,Gerzensee,SVP / UDC,Ja / Oui,2018-06-04 15:47:00,137.0,1.0,1.0,Abstimmung mit Namensliste / Procès-verbal de ...,Grossrat / le grand conseil,11.0,#2017.RRGR.459_0,Eröffnung der Sitzung durch das älteste der an...,Sommersession 2018.xlsx,4.04.2018,rathaus


In [6]:
# Im Datenatz fehlen die Vornamen. Diese werden eingefügt. 
df_temp = df_rathaus[['nachname', 'wohnort', 'partei']].drop_duplicates()
df_temp = pd.merge(df_temp, df_grossrat, left_on=['nachname', 'wohnort'], right_on=['nachname', 'wohnort'], how='left')
df_temp = df_temp[['nachname', 'vorname', 'wohnort', 'partei_x']]
df_temp.columns = ['nachname', 'vorname', 'wohnort', 'partei']
df_temp['test'] = df_temp['nachname'] + df_temp['wohnort'] + df_temp['partei']

# Leider wurden -- z.B. wegen unterschiedlichen Ortsangaben in den beiden Datensätzen -- nicht alle
# Vornamen gefunden. Deshalb wird nun ein ungenaues Matching durchgeführt. 
df_grossrat['test'] = df_grossrat[['nachname', 'wohnort', 'partei']].sum(axis=1).tolist()

k_list = list()
for i, r in df_temp[df_temp['vorname'].isnull()].iterrows():
    best = process.extract(r['test'], df_grossrat['test'].tolist())
    
    print('Bestehende Infos:' + r['nachname'], ' | ', r['partei'], ' | ', r['wohnort'])
    print('Ergänzt durch: ' + df_grossrat[df_grossrat['test'] == best[0][0]]['vorname'].values[0], df_grossrat[df_grossrat['test'] == best[0][0]]['nachname'].values[0])
    k = input('Eingabe, wenn korrekt, sonst "n" ')
    
    if k == '':
        df_temp.at[i, 'vorname'] = df_grossrat[df_grossrat['test'] == best[0][0]]['vorname'].values[0]
    else:
        k_list.append(r['nachname'])

    
del df_grossrat['test']
del df_temp['test']

# Die Tabelle mit den vollen Namen wird dem Originaldatensatz hinzugefügt. 
df_temp = pd.merge(df_rathaus, df_temp[['nachname', 'vorname', 'partei']], left_on=['nachname', 'partei'], right_on=['nachname', 'partei'], how='left')
df_temp = df_temp.drop_duplicates(subset=['nachname', 'wohnort', 'stimme', 'traktandum', 'dateiname', 'register'])

Bestehende Infos:Aebischer  |  SVP / UDC  |  Riffenmatt
Ergänzt durch: Verena Aebischer
Eingabe, wenn korrekt, sonst "n" 
Bestehende Infos:Amstutz  |  SVP / UDC  |  Schwanden-Sigriswil
Ergänzt durch: Madeleine Amstutz
Eingabe, wenn korrekt, sonst "n" 
Bestehende Infos:Graber  |  SVP / UDC  |  La Neuveville
Ergänzt durch: Anne-Caroline Graber
Eingabe, wenn korrekt, sonst "n" 
Bestehende Infos:Gschwend-Pieren  |  SVP / UDC  |  Lyssach
Ergänzt durch: Andrea Gschwend-Pieren
Eingabe, wenn korrekt, sonst "n" 
Bestehende Infos:Sutter  |  SVP / UDC  |  Langnau i.E.
Ergänzt durch: Walter Sutter
Eingabe, wenn korrekt, sonst "n" 
Bestehende Infos:Buri  |  SVP / UDC  |  Hasle bei Burgdorf
Ergänzt durch: Roland Näf
Eingabe, wenn korrekt, sonst "n" n
Bestehende Infos:Gabi Schönenberger  |  SP-JUSO-PSA / PS-JS-PSA  |  Schwarzenburg
Ergänzt durch: Sarah Gabi Schönenberger
Eingabe, wenn korrekt, sonst "n" 
Bestehende Infos:Näf  |  SP-JUSO-PSA / PS-JS-PSA  |  Muri
Ergänzt durch: Roland Näf
Eingabe, wenn

In [7]:
k_list

['Buri', 'Bauen', 'Gnägi', 'Kipfer', 'Riesen', 'Riesen']

In [8]:
# Einige manuelle Korrekturen.

for i, r in df_temp[df_temp['vorname'].isnull()].iterrows():
    if r['nachname'] == 'Buri' and 'Hasle' in r['wohnort']:
        df_temp.at[i, 'vorname'] = 'Urs'
        df_temp.at[i, 'wohnort'] = 'Hasle b. B.'
    elif r['nachname'] == 'Kipfer':
        df_temp.at[i, 'vorname'] = 'Hans'
        df_temp.at[i, 'wohnort'] = 'Münsingen'
    elif r['nachname'] == 'Bauer':
        df_temp.at[i, 'vorname'] = 'Tanja'
        df_temp.at[i, 'wohnort'] = 'Wabern'
    elif r['nachname'] == 'Wüthrich':
        df_temp.at[i, 'vorname'] = 'Adrian'
    elif r['nachname'] == 'Bauen':
        df_temp.at[i, 'vorname'] = 'Antonio'
        df_temp.at[i, 'wohnort'] = 'Bern'
    elif r['nachname'] == 'Gnägi':
        df_temp.at[i, 'vorname'] = 'Jan'
        df_temp.at[i, 'wohnort'] = 'Aarberg'
        df_temp.at[i, 'partei'] = 'Die Mitte'
    elif r['nachname'] == 'Grimm':
        df_temp.at[i, 'vorname'] = 'Christoph'
        df_temp.at[i, 'wohnort'] = 'Burgdorf'
    elif r['nachname'] == 'Riesen' and r['partei'] == 'PSA':
        df_temp.at[i, 'vorname'] = 'Maurane'
        df_temp.at[i, 'wohnort'] = 'La Neuveville'        

In [9]:
# Fehlende Spalten werden eingefügt. Datensatz wird richtig gegliedert. 
df_temp['res_keine_stimme'] = np.NaN
df_temp = df_temp[['nachname', 'vorname', 'wohnort', 'partei', 'datetime', 'traktandum', 'stimme', 'res_ja', 'res_nein', 'res_enthaltung', 'res_keine_stimme', 'anlage', 'dateiname', 'register']]
df_temp.columns = ['nachname', 'vorname', 'wohnort', 'partei', 'datetime', 'geschäftsname', 'stimme', 'res_ja', 'res_nein', 'res_enthaltung', 'res_keine_stimme', 'anlage', 'dateiname', 'register']
df_temp = df_temp.drop_duplicates()
df_rathaus = df_temp.copy()

In [23]:
# Abspeichern der Daten. 
df_rathaus.to_csv('daten/rathaus.csv')

## BEA-Session 1

In [249]:
# Die Excel-Dateien (Version 1) werden eingelesen. 
df_bea_1 = pd.DataFrame()

for d in tqdm(format_bea_v1):

    # Durch die Register loopen...
    register = pd.ExcelFile('daten/bea1/' + d).sheet_names

    for r in register:
    
        # Die Metadaten werden ausgelesen. 
        df_temp = pd.read_excel('daten/bea1/' + d, sheet_name=r)
        temp_geschäftsname = ' '.join([x for x in pd.read_excel('daten/bea1/' + d, sheet_name=r).columns if 'Unnamed' not in x])
        temp_datetime = df_temp[df_temp.iloc[:,0] == 'Date and Time:'].iloc[0,1]
        temp_ja = df_temp[df_temp.iloc[:,0] == 'Yes votes:'].iloc[0,1]
        temp_nein = df_temp[df_temp.iloc[:,0] == 'No votes:'].iloc[0,1]
        temp_enthaltungen = df_temp[df_temp.iloc[:,0] == 'Abstentions:'].iloc[0,1]
        temp_keine_stimme = df_temp[df_temp.iloc[:,0] == 'Not voting:'].iloc[0,1]

        # Der Abstimmungsresultate werden eingelesen. 
        df_temp_roh = pd.read_excel('daten/bea1/' + d, sheet_name=r, skiprows=12)
        df_temp_roh = df_temp_roh.rename(columns={'Anrede': 'TED'})
        
        # Die Ja, Nein, Enthaltungen sind in unter einander stehenden Tabellen aufgeführt.
        # Deshalb muss mit Subsets gearbeitet werden. 
        schnittmarke_nein = df_temp_roh[df_temp_roh['TED'] == 'Against'].iloc[0].name
        schnittmarke_enthaltung = df_temp_roh[df_temp_roh['TED'] == 'Abstentions'].iloc[0].name

        df_temp = pd.DataFrame()
        for v in ['ja', 'nein', 'enthaltung']:
            if v == 'ja':
                df_temp1 = df_temp_roh.copy().iloc[:schnittmarke_nein]
                df_temp1['stimme'] = 'Ja / Oui'
            elif v == 'nein':
                df_temp1 = df_temp_roh.copy().iloc[schnittmarke_nein + 1:schnittmarke_enthaltung]
                df_temp1['stimme'] = 'Nein / Non'
            elif v == 'enthaltung':
                df_temp1 = df_temp_roh.copy().iloc[schnittmarke_enthaltung + 1:]
                df_temp1['stimme'] = 'Enthaltungen / Abstentions'
            df_temp = df_temp.append(df_temp1)

        # Die Metadaten werden hinzugefügt.
        df_temp['geschäftsname'] = temp_geschäftsname
        df_temp['datetime'] = temp_datetime
        df_temp['res_ja'] = temp_ja
        df_temp['res_nein'] = temp_nein
        df_temp['enthaltungen'] = temp_enthaltungen
        df_temp['keine_stimme'] = temp_keine_stimme
        df_temp['dateiname'] = d
        df_temp['register'] = r
        
        # In den Exporten der Herbstsession sind plötzlich die Spaltennamen in Grossbuchstaben. Das wird angepasst.
        df_temp.columns = [x.lower() for x in df_temp.columns]

        # Die neuen Daten werden dem Datensatz angehängt. 
        df_bea_1 = df_bea_1.append(df_temp)

100%|██████████| 22/22 [05:48<00:00, 15.84s/it]


In [250]:
df_backup = df_bea_1.copy()

In [351]:
df_bea_1 = df_backup.copy()

In [352]:
# Die Daten werden bereinigt. 
df_bea_1 = df_bea_1[~df_bea_1['partei'].isnull()]
df_bea_1 = df_bea_1[df_bea_1['name'] != 'Name/Nom']
df_bea_1 = df_bea_1[df_bea_1['name'] != 'Name']
df_bea_1 = df_bea_1[df_bea_1['name'] != 'NAME']

df_bea_1 = df_bea_1[~df_bea_1['geschäftsname'].str.lower().str.contains('test')]
df_bea_1.reset_index(inplace=True, drop=True)

# In einigen Namen gibts Sonderzeichen, die das nachträgliche Matching mit dem Personendatensatz verhindern. 
df_bea_1['name'] = df_bea_1['name'].str.replace('\xa0', '').str.strip()
df_bea_1['vornamen'] = df_bea_1['vornamen'].str.replace('\xa0', '').str.strip()

In [353]:
# Einige Korrekturen. 
df_bea_1.reset_index(inplace=True, drop=True)

# Ulrich Frutiger heisst in der Personendatenbank Ueli Frutiger. 
for i, r in df_bea_1[df_bea_1['name'] == 'Frutiger'].iterrows():
    df_bea_1.at[i, 'vornamen'] = 'Ueli'

# Bei Roulet Romy sind die Vornamen und Namen durcheinander geraten...
for i, r in df_bea_1[df_bea_1['name'].str.contains('Roulet')].iterrows():
    df_bea_1.at[i, 'name'] = 'Roulet Romy'
    df_bea_1.at[i, 'vornamen'] = 'Sandra'

# Bei Stefan Müller fehlt der zweite Vorname.
for i, r in df_bea_1[(df_bea_1['name'] == 'Müller') & (df_bea_1['vornamen'] == 'Stefan')].iterrows():
    df_bea_1.at[i, 'vornamen'] = 'Stefan Bänz'

# Bei einigen Daten ist heisst die Vornamensspalte "vornamen" statt "vorname". 
df_bea_1['vornamen'].update(df_bea_1['vorname'])

# Eine nicht plausible Abstimmung mit bloss ganz wenigen Teilnehmenden wird ausgefiltert. 
df_temp = df_bea_1.groupby(by=['datetime', 'geschäftsname', 'register', 'dateiname'])['stimme'].count().to_frame()
n = df_temp[df_temp['stimme'] < 10].reset_index()['geschäftsname'][0]
df_bea_1 = df_bea_1[df_bea_1['geschäftsname'] != n]

df_bea_1['anlage'] = 'bea1'
df_bea_1.head()

,ted,name,vornamen,partei,pultnummer und abstimmungseinheit,stimme,geschäftsname,datetime,res_ja,res_nein,enthaltungen,keine_stimme,dateiname,register,vorname,sitzplatz,anlage
0,2,Müller,Mathias,SVP,2,Ja / Oui,Motion: 2.Traktandum / Point de l’ordre du jou...,2020-06-08 13:58:45,102,46,4,8,04) 8.06.2020 Sommersession 2020.xlsx,2,NaN,NaN,bea1
1,156,Baumann-Berger,Katharina,EDU,156,Ja / Oui,Motion: 2.Traktandum / Point de l’ordre du jou...,2020-06-08 13:58:45,102,46,4,8,04) 8.06.2020 Sommersession 2020.xlsx,2,NaN,NaN,bea1
2,103,Moser,Peter,FDP,103,Ja / Oui,Motion: 2.Traktandum / Point de l’ordre du jou...,2020-06-08 13:58:45,102,46,4,8,04) 8.06.2020 Sommersession 2020.xlsx,2,NaN,NaN,bea1
3,124,Mühlemann,Andreas,BDP,124,Ja / Oui,Motion: 2.Traktandum / Point de l’ordre du jou...,2020-06-08 13:58:45,102,46,4,8,04) 8.06.2020 Sommersession 2020.xlsx,2,NaN,NaN,bea1
4,44,Speiser-Niess,Anne,SVP,44,Ja / Oui,Motion: 2.Traktandum / Point de l’ordre du jou...,2020-06-08 13:58:45,102,46,4,8,04) 8.06.2020 Sommersession 2020.xlsx,2,NaN,NaN,bea1


In [350]:
'''# Bei einigen Daten fehlen die Angaben zum Vornamen. Wir ergänzen diesen aus dem Grossratsdatensatz. 
korrdict = dict()
for i, r in df_bea_1[df_bea_1['vornamen'].isnull()].iterrows(): 
    df_temp = df_grossrat[df_grossrat['nachname'] == r['name']]
    if len(df_temp) == 1:
        df_bea_1.at[i, 'vornamen'] = df_temp.iloc[0]['vorname']
    elif len(df_temp) > 1:
        df_temp2 = df_grossrat[(df_grossrat['nachname'] == r['name']) & (df_grossrat['partei'] == r['partei'])]
        if len(df_temp2) == 1:
            df_bea_1.at[i, 'vornamen'] = df_temp2.iloc[0]['vorname']
        else:
            if r['name'] in korrdict.keys():
                df_bea_1.at[i, 'vornamen'] = korrdict[r['name']]
            else:
                print(r)
                vn = input('Wie lautet der korrekte Vorname? ')
                df_bea_1.at[i, 'vornamen'] = vn
                korrdict[r['name']] = vn'''

"# Bei einigen Daten fehlen die Angaben zum Vornamen. Wir ergänzen diesen aus dem Grossratsdatensatz. \nkorrdict = dict()\nfor i, r in df_bea_1[df_bea_1['vornamen'].isnull()].iterrows(): \n    df_temp = df_grossrat[df_grossrat['nachname'] == r['name']]\n    if len(df_temp) == 1:\n        df_bea_1.at[i, 'vornamen'] = df_temp.iloc[0]['vorname']\n    elif len(df_temp) > 1:\n        df_temp2 = df_grossrat[(df_grossrat['nachname'] == r['name']) & (df_grossrat['partei'] == r['partei'])]\n        if len(df_temp2) == 1:\n            df_bea_1.at[i, 'vornamen'] = df_temp2.iloc[0]['vorname']\n        else:\n            if r['name'] in korrdict.keys():\n                df_bea_1.at[i, 'vornamen'] = korrdict[r['name']]\n            else:\n                print(r)\n                vn = input('Wie lautet der korrekte Vorname? ')\n                df_bea_1.at[i, 'vornamen'] = vn\n                korrdict[r['name']] = vn"

In [355]:
# Fehlende Felder (Wohnort) werden aus dem Grossrats-Datensatz ergänzt.  
df_bea_1 = pd.merge(df_bea_1, df_grossrat[['nachname', 'vorname', 'wohnort']], left_on=['name', 'vornamen'], right_on=['nachname', 'vorname'], how='left')

df_bea_1 = df_bea_1.drop_duplicates(subset=df_bea_1.columns, keep='last', ignore_index=True)
df_bea_1.reset_index(inplace=True, drop=True)

In [232]:
'''# Einige Korrekturen: 
for i, r in df_bea_1[df_bea_1['wohnort'].isnull()].iterrows():
    if r['name'] == 'Müller' and r['vornamen'] == 'Stefan':
        df_bea_1.at[i, 'wohnort'] = 'Innerberg'
        df_bea_1.at[i, 'vornamen'] = 'Stefan Bänz'
    elif r['name'] == 'Marti':
        df_bea_1.at[i, 'vornamen'] = ''
    elif r['name'] == 'Roulet':
        df_bea_1.at[i, 'name'] = 'Roulet Romy'
        df_bea_1.at[i, 'vornamen'] = 'Sandra'
        df_bea_1.at[i, 'wohnort'] = 'Malleray'
    elif r['name'] == 'Roulet Romy ':
        df_bea_1.at[i, 'name'] = 'Roulet Romy'
        df_bea_1.at[i, 'vornamen'] = 'Sandra'
        df_bea_1.at[i, 'wohnort'] = 'Malleray'
    elif r['name'] == 'Geissbühler-Strupler':
        df_bea_1.at[i, 'wohnort'] = 'Herrenschwanden'
    elif r['name'] == 'Müller' and r['vornamen'] == 'Stefan':
        df_bea_1.at[i, 'wohnort'] = 'Innerberg'
        df_bea_1.at[i, 'vornamen'] = 'Stefan Bänz'
    elif r['name'] == 'Rothenbühler ' and r['vornamen'] == 'Jürg':
        df_bea_1.at[i, 'wohnort'] = 'Rüderswil'
    elif r['name'] == 'Bichsel ' and r['vornamen'] == 'Alfons':
        df_bea_1.at[i, 'wohnort'] = 'Merligen'
        df_bea_1.at[i, 'name'] = 'Bichsel'
    elif r['name'] == 'Bichsel':
        df_bea_1.at[i, 'wohnort'] = 'Merligen'
        df_bea_1.at[i, 'nachname'] = 'Bichsel'
 
    elif r['name'] == 'Rothenbühler ':
        df_bea_1.at[i, 'vorname'] = 'Jürg'
        df_bea_1.at[i, 'wohnort'] = 'Rüderswil'

'''

In [357]:
df_bea_1 = df_bea_1[['nachname', 'vornamen', 'wohnort', 'partei', 'datetime', 'geschäftsname', 'stimme', 'res_ja', 'res_nein', 'enthaltungen', 'keine_stimme', 'anlage', 'dateiname', 'register']]
df_bea_1.columns = ['nachname', 'vorname', 'wohnort', 'partei', 'datetime', 'geschäftsname', 'stimme', 'res_ja', 'res_nein', 'res_enthaltung', 'res_keine_stimme', 'anlage', 'dateiname', 'register']
   

In [359]:
# Abspeichern der Daten. 
df_bea_1.to_csv('daten/bea1.csv')

## BEA-Session 2

In [41]:
# Die Excel-Dateien (Version 2) werden eingelesen. 
df_bea_2 = pd.DataFrame()

# Durch die Dateien loopen. 
for d in tqdm(format_bea_v2):
       
    # Durch die Register loopen.
    register = pd.ExcelFile('daten/bea2/' + d).sheet_names

    for r in register: 
        df_temp = pd.read_excel('daten/bea2/' + d, sheet_name=r)
        df_temp['dateiname'] = d
        df_temp['register'] = r
        df_bea_2 = df_bea_2.append(df_temp)

# Platzhalter für fehlende Spalten werden eingefügt. 
df_bea_2['wohnort'] = ''
df_bea_2['anlage'] = 'bea2'

# Die Resultate der einzelnen Abstimmungen werden berechnet.
df_res = df_bea_2.groupby(by=['Headline Text', 'Creation Date', 'Current Voting ID', 'Run Name'])['Choice Text'].value_counts().to_frame().unstack()
df_res.columns = [x[1] for x in df_res.columns]
df_res.columns = ['res_abwesend', 'res_enthaltung', 'res_ja', 'res_nein']
df_res = df_res.reset_index()

# Die Angaben zu den Resultaten werden dem Datensatz hinzugefügt. 
df_bea_2 = pd.merge(df_bea_2, df_res, left_on=['Headline Text', 'Creation Date', 'Current Voting ID', 'Run Name'], right_on=['Headline Text', 'Creation Date', 'Current Voting ID', 'Run Name'])

# Eine Datetime-Spalte wird hinzugefügt. 
for i, r in df_bea_2.iterrows():
    if str(r['Received Time']) == 'nan':
        df_bea_2.at[i, 'datetime'] = str(r['Creation Date'])
    else:
        df_bea_2.at[i, 'datetime'] = str(r['Creation Date'])[:-9] + ' ' + str(r['Received Time'])

# Um ganz sicher keine Abstimmungen zu verlieren, wird der "Run Name" dem "Headline Text" angehängt. 
df_bea_2['Headline Text'] = df_bea_2['Headline Text'] + '_' + df_bea_2['Run Name'].str.replace('Run ', '')

# Ein Datetime-Feld wird eingefügt. 
df_bea_2['datetime'] = [pd.to_datetime(x) for x in df_bea_2['datetime']]

# In einigen Namen gibts Sonderzeichen, die das nachträgliche Matching mit dem Personendatensatz verhindern. 
df_bea_2['Name'] = df_bea_2['Name'].str.replace('\xa0', '')

# Duplikate werden entfernt. 
df_bea_2 = df_bea_2.drop_duplicates(subset=['Name', 'Creation Date', 'Received Time', 'Choice Text', 'Headline Text', 'dateiname', 'register'])
df_bea_2.head()

100%|██████████| 9/9 [00:48<00:00,  5.39s/it]


,Unnamed: 0,Current Voting ID,Headline Text,Handset ID,Vote Group,Name,Received Text,Choice Text,Creation Date,Received Time,Run Name,dateiname,register,wohnort,anlage,res_abwesend,res_enthaltung,res_ja,res_nein,datetime
0,NaN,259,99 2020.RRGR.132_21,801,SVP/UDC,Josi Barbara,2,Nein/Non,2021-03-18,09:46:12,Run 21,08) 18.03.21 Frühlingssession 2021.xlsx,Tabelle1,,bea2,7.0,7.0,50.0,96.0,2021-03-18 09:46:12
1,NaN,259,99 2020.RRGR.132_21,802,SVP/UDC,Müller Mathias,2,Nein/Non,2021-03-18,09:46:11,Run 21,08) 18.03.21 Frühlingssession 2021.xlsx,Tabelle1,,bea2,7.0,7.0,50.0,96.0,2021-03-18 09:46:11
2,NaN,259,99 2020.RRGR.132_21,803,SVP/UDC,Freudiger Patrick,2,Nein/Non,2021-03-18,09:46:13,Run 21,08) 18.03.21 Frühlingssession 2021.xlsx,Tabelle1,,bea2,7.0,7.0,50.0,96.0,2021-03-18 09:46:13
3,NaN,259,99 2020.RRGR.132_21,804,SVP/UDC,Bichsel Daniel,2,Nein/Non,2021-03-18,09:46:03,Run 21,08) 18.03.21 Frühlingssession 2021.xlsx,Tabelle1,,bea2,7.0,7.0,50.0,96.0,2021-03-18 09:46:03
4,NaN,259,99 2020.RRGR.132_21,805,SVP/UDC,Feuz Alexander,2,Nein/Non,2021-03-18,09:46:05,Run 21,08) 18.03.21 Frühlingssession 2021.xlsx,Tabelle1,,bea2,7.0,7.0,50.0,96.0,2021-03-18 09:46:05


In [42]:
# Fehlende Felder werden aus dem Grossrats-Datensatz ergänzt.  
df_grossrat['name-vorname'] = df_grossrat['nachname'] + ' ' + df_grossrat['vorname']
df_bea_2['Name'] = df_bea_2['Name'].apply(lambda x: x.replace('  ', ' ').strip())
df_bea_2 = pd.merge(df_bea_2, df_grossrat[['nachname', 'vorname', 'wohnort', 'name-vorname']], left_on='Name', right_on='name-vorname', how='left')
df_bea_2 = df_bea_2.drop_duplicates(subset=['Name', 'Creation Date', 'Received Time', 'Choice Text', 'Headline Text', 'dateiname', 'register'])
df_bea_2.reset_index(inplace=True, drop=True)

In [43]:
# Einige Korrekturen: 
for i, r in df_bea_2[df_bea_2['nachname'].isnull()].iterrows():
    if 'Roulet' in r['Name']:
        df_bea_2.at[i, 'nachname'] = 'Roulet Romy'
        df_bea_2.at[i, 'vorname'] = 'Sandra'
        df_bea_2.at[i, 'wohnort'] = 'Malleray'
    elif 'Geissbühler-Strupler' in r['Name']:
        df_bea_2.at[i, 'nachname'] = 'Geissbühler-Strupler'
        df_bea_2.at[i, 'vorname'] = 'Sabina'
        df_bea_2.at[i, 'wohnort'] = 'Herrenschwanden'
    elif 'Müller Stefan' in r['Name']:
        df_bea_2.at[i, 'wohnort'] = 'Innerberg'
        df_bea_2.at[i, 'vorname'] = 'Stefan Bänz'
        df_bea_2.at[i, 'nachname'] = 'Müller'
    elif 'Sager' in r['Name']:
        df_bea_2.at[i, 'wohnort'] = 'Herzogenbuchsee'
        df_bea_2.at[i, 'vorname'] = 'Ruth'
        df_bea_2.at[i, 'nachname'] = 'Sager'
    elif 'Bichsel' in r['Name']:
        df_bea_2.at[i, 'wohnort'] = 'Merligen'
        df_bea_2.at[i, 'nachname'] = 'Bichsel'
        df_bea_2.at[i, 'vorname'] = 'Alfons'
    elif 'Rothenbühler' in r['Name']:
        df_bea_2.at[i, 'wohnort'] = 'Rüderswil'
        df_bea_2.at[i, 'nachname'] = 'Rothenbühler'
        df_bea_2.at[i, 'vorname'] = 'Alfons'
    elif 'Schori' in r['Name'] and 'Beat' in r['Name']:
        df_bea_2.at[i, 'wohnort'] = 'Bern'
        df_bea_2.at[i, 'nachname'] = 'Schori'
        df_bea_2.at[i, 'vorname'] = 'Beat'

In [44]:
df_bea_2 = df_bea_2[['nachname', 'vorname', 'wohnort_x', 'Vote Group', 'datetime', 'Headline Text', 'Choice Text', 'res_ja', 'res_nein', 'res_enthaltung', 'res_abwesend', 'anlage', 'dateiname', 'register']]
df_bea_2.columns = ['nachname', 'vorname', 'wohnort', 'partei', 'datetime', 'geschäftsname', 'stimme', 'res_ja', 'res_nein', 'res_enthaltung', 'res_keine_stimme', 'anlage', 'dateiname', 'register']
del df_grossrat['name-vorname']
df_bea_2.to_csv('daten/bea2.csv')

## Bea-Session 3 (Festhalle)

In [45]:
# Die Excel-Dateien (Version 2) werden eingelesen. 
df_bea_3 = pd.DataFrame()

# Durch die Dateien loopen. 
for d in tqdm(format_bea_v3):
       
    # Durch die Register loopen.
    register = pd.ExcelFile('daten/bea3/' + d).sheet_names

    for r in register: 
        df_temp = pd.read_excel('daten/bea3/' + d, sheet_name=r)
        df_temp['dateiname'] = d
        df_temp['register'] = r
        df_bea_3 = df_bea_3.append(df_temp)

# Platzhalter für fehlende Spalten werden eingefügt. 
df_bea_3['wohnort'] = ''
df_bea_3['anlage'] = 'bea3'

# Die Resultate der einzelnen Abstimmungen werden berechnet.
df_res = df_bea_3.groupby(by=['Headline Text', 'Creation Date', 'Current Voting ID', 'Run Name'])['Choice Text'].value_counts().to_frame().unstack()
df_res.columns = [x[1] for x in df_res.columns]
df_res.columns = ['res_abwesend', 'res_enthaltung', 'res_ja', 'res_nein']
df_res = df_res.reset_index()

# Die Angaben zu den Resultaten werden dem Datensatz hinzugefügt. 
df_bea_3 = pd.merge(df_bea_3, df_res, left_on=['Headline Text', 'Creation Date', 'Current Voting ID', 'Run Name'], right_on=['Headline Text', 'Creation Date', 'Current Voting ID', 'Run Name'])

# Eine Datetime-Spalte wird hinzugefügt. 
for i, r in df_bea_3.iterrows():
    if str(r['Received Time']) == 'nan':
        df_bea_3.at[i, 'datetime'] = str(r['Creation Date'])
    else:
        df_bea_3.at[i, 'datetime'] = str(r['Creation Date'])[:-9] + ' ' + str(r['Received Time'])

# Um ganz sicher keine Abstimmungen zu verlieren, wird der "Run Name" dem "Headline Text" angehängt. 
df_bea_3['Headline Text'] = df_bea_3['Headline Text'] + '_' + df_bea_3['Run Name'].str.replace('Run ', '')

# Ein Datetime-Feld wird eingefügt. 
df_bea_3['datetime'] = [pd.to_datetime(x) for x in df_bea_3['datetime']]

# In einigen Namen gibts Sonderzeichen, die das nachträgliche Matching mit dem Personendatensatz verhindern. 
df_bea_3['Name'] = df_bea_3['Name'].str.replace('\xa0', '')

# Duplikate werden entfernt. 
df_bea_3 = df_bea_3.drop_duplicates(subset=['Name', 'Creation Date', 'Received Time', 'Choice Text', 'Headline Text', 'dateiname', 'register'])
df_bea_3.head()

# Leere Einträge werden entfernt. 
df_bea_3 = df_bea_3[df_bea_3['Name'] != 'leer']
df_bea_3 = df_bea_3[~df_bea_3['Name'].isnull()]

100%|██████████| 22/22 [00:16<00:00,  1.32it/s]


In [46]:
# Fehlende Felder werden aus dem Grossrats-Datensatz ergänzt.  
df_grossrat['name-vorname'] = df_grossrat['nachname'] + ' ' + df_grossrat['vorname']
df_bea_3['Name'] = df_bea_3['Name'].apply(lambda x: x.replace('  ', ' ').strip())
df_bea_3 = pd.merge(df_bea_3, df_grossrat[['nachname', 'vorname', 'wohnort', 'name-vorname']], left_on='Name', right_on='name-vorname', how='left')
df_bea_3 = df_bea_3.drop_duplicates(subset=['Name', 'Creation Date', 'Received Time', 'Choice Text', 'Headline Text', 'dateiname', 'register'])
df_bea_3.reset_index(inplace=True, drop=True)

In [47]:
# Einige Korrekturen: 
for i, r in df_bea_3[df_bea_3['nachname'].isnull()].iterrows():
    if 'Roulet' in r['Name']:
        df_bea_3.at[i, 'nachname'] = 'Roulet Romy'
        df_bea_3.at[i, 'vorname'] = 'Sandra'
        df_bea_3.at[i, 'wohnort'] = 'Malleray'
    elif 'Geissbühler-Strupler' in r['Name']:
        df_bea_3.at[i, 'nachname'] = 'Geissbühler-Strupler'
        df_bea_3.at[i, 'vorname'] = 'Sabina'
        df_bea_3.at[i, 'wohnort'] = 'Herrenschwanden'
    elif 'Müller Stefan' in r['Name']:
        df_bea_3.at[i, 'wohnort'] = 'Innerberg'
        df_bea_3.at[i, 'vorname'] = 'Stefan Bänz'
        df_bea_3.at[i, 'nachname'] = 'Müller'
    elif 'Sager' in r['Name']:
        df_bea_3.at[i, 'wohnort'] = 'Herzogenbuchsee'
        df_bea_3.at[i, 'vorname'] = 'Ruth'
        df_bea_3.at[i, 'nachname'] = 'Sager'
    elif 'Bichsel' in r['Name']:
        df_bea_3.at[i, 'wohnort'] = 'Merligen'
        df_bea_3.at[i, 'nachname'] = 'Bichsel'
        df_bea_3.at[i, 'vorname'] = 'Alfons'
    elif 'Rothenbühler' in r['Name']:
        df_bea_3.at[i, 'wohnort'] = 'Rüderswil'
        df_bea_3.at[i, 'nachname'] = 'Rothenbühler'
        df_bea_3.at[i, 'vorname'] = 'Alfons'
    elif 'Schori' in r['Name'] and 'Beat' in r['Name']:
        df_bea_3.at[i, 'wohnort'] = 'Bern'
        df_bea_3.at[i, 'nachname'] = 'Schori'
        df_bea_3.at[i, 'vorname'] = 'Beat'
    elif 'Elsaesser' in r['Name'] and 'Michael' in r['Name']:
        df_bea_3.at[i, 'wohnort'] = 'Kirchberg'
        df_bea_3.at[i, 'nachname'] = 'Elsaesser'
        df_bea_3.at[i, 'vorname'] = 'Michael'
    elif 'Remund' in r['Name'] and 'Jan' in r['Name']:
        df_bea_3.at[i, 'wohnort'] = 'Mittelhäusern'
        df_bea_3.at[i, 'nachname'] = 'Remund'
        df_bea_3.at[i, 'vorname'] = 'Jan'

In [48]:
# Da die Namen teilweise anders geschrieben werden (z.B. doppelte Zwischenräume) wird ein ungenaues Matching gemacht. 
for i, r in tqdm(df_bea_3[df_bea_3['nachname'].isnull()].iterrows(), total=len(df_bea_3[df_bea_3['nachname'].isnull()])):
    nn = process.extractOne(r['Name'], df_grossrat['name-vorname'], score_cutoff = 80)
    df_bea_3.at[i, 'vorname'] = df_grossrat[df_grossrat['name-vorname'] == nn[0]].iloc[0]['vorname']
    df_bea_3.at[i, 'nachname'] = df_grossrat[df_grossrat['name-vorname'] == nn[0]].iloc[0]['nachname']

df_bea_3 = df_bea_3[['nachname', 'vorname', 'wohnort_x', 'Vote Group', 'datetime', 'Headline Text', 'Choice Text', 'res_ja', 'res_nein', 'res_enthaltung', 'res_abwesend', 'anlage', 'dateiname', 'register']]
df_bea_3.columns = ['nachname', 'vorname', 'wohnort', 'partei', 'datetime', 'geschäftsname', 'stimme', 'res_ja', 'res_nein', 'res_enthaltung', 'res_keine_stimme', 'anlage', 'dateiname', 'register']
del df_grossrat['name-vorname']

# Abspeichern der Daten. 
df_bea_3.to_csv('daten/bea3.csv')

0it [00:00, ?it/s]


## Harmonisieren und zusammenfügen der Daten 

In [170]:
# Neu einlesen der Daten. 
df_rathaus = pd.read_csv('daten/rathaus.csv', index_col='Unnamed: 0')
df_bea_1 = pd.read_csv('daten/bea1.csv', index_col='Unnamed: 0')
df_bea_2 = pd.read_csv('daten/bea2.csv', index_col='Unnamed: 0')
df_bea_3 = pd.read_csv('daten/bea3.csv', index_col='Unnamed: 0')

2021-06-28 13:36:54,225 [13187] WARNING  py.warnings:109: [JupyterRequire] /usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



In [171]:
# Zusammenfügen der Daten. 
df = df_rathaus.append(df_bea_1)
df = df.append(df_bea_2)
df = df.append(df_bea_3)
df.reset_index(drop=True, inplace=True)

In [172]:
# Einige Retouchen. 

# Einträge, die die Tabellenbeschriftungen enthalten, werden entfernt. 
df = df[df['partei'] != 'Partei']

# Die Datetime-Spalte wird ins Datetime-Format überführt. 
df['datetime'] = pd.to_datetime(df['datetime'], errors='raise')

# Die Angaben zur Stimme (ja, nein, Enthaltung etc. werden harmonisiert.)
df = df.dropna(subset=['stimme'])

def janein(stimme):
    if 'ja' in stimme.lower():
        return 'ja'
    elif 'nein' in stimme.lower():
        return 'nein'
    elif 'enthaltung' in stimme.lower():
        return 'enthaltung'
    elif 'abwesend' in stimme.lower() or '-' in stimme.lower():
        return 'abwesend'
    
df['stimme'] = df['stimme'].apply(janein)

In [173]:
# Die Namen der Parteien sind unterschiedlich erfasst. Sie werden harmonisiert. 
parteien_dict = {'SVP / UDC': 'SVP', 'SP-JUSO-PSA / PS-JS-PSA': 'SP', 'FDP / PLR': 'FDP',
                 'Grüne / Les Verts': 'Grüne', 'BDP / PBD': 'Die Mitte', 'glp / pvl': 'glp', 
                 'EVP / PEV': 'EVP', 'EDU / UDF': 'EDU', 'CVP / PDC': 'Die Mitte', 'BDP': 'Die Mitte',
                 'PLR': 'FDP', 'PSA': 'SP', 'CVP': 'Die Mitte', 'PS': 'SP', 'les Verts': 'Grüne',
                 'UDC': 'SVP', 'PEV': 'EVP', 'PDC': 'Die Mitte', 'SVP/UDC': 'SVP', 'FDP/PLR': 'FDP',
                 'Grüne/les Verts': 'Grüne', 'glp/plv': 'glp', 'EVP/PEV': 'EVP', 'EDU/UDF': 'EDU',
                 'Grünliberale': 'glp', 'Les Verts': 'Grüne', 'AL': 'Grüne'}
def harmos(partei):
    if partei in parteien_dict.keys(): 
        return parteien_dict[partei]
    else:
        return partei

df['partei'] = df['partei'].apply(harmos)

In [174]:
# Seiler wird teilweise als "parteilos", teilweise als "fraktionslos" bezeichnet. Das wird harmonisiert.
for i, r in tqdm(df[df['nachname'] == 'Seiler'].iterrows(), total=len(df[df['nachname'] == 'Seiler'])):
    if r['nachname'] == 'Seiler' and r['vorname'] == 'Michel' and r['partei'] == 'parteilos':
        df.at[i, 'partei'] = 'fraktionslos'

100%|██████████| 2255/2255 [00:00<00:00, 9410.48it/s]


In [175]:
# Einige Namen und Vornamen enthalten zu viele Leerzeichen. 
df['vorname'] = df['vorname'].str.strip()
df['vorname'] = df['vorname'].str.replace('  ', ' ')
df['nachname'] = df['nachname'].str.strip()
df['nachname'] = df['nachname'].str.replace('  ', ' ')

In [176]:
# Die Infos zum Wohnort und zum Wahlkreis und zur Sprache werden aus dem Mitglieder-Verzeichnis hinzugefügt. 
df = pd.merge(df, df_grossrat[['nachname', 'vorname', 'wohnort', 'wahlkreis', 'sprache', 'geschlecht', 'geburtstag', 'eintritt', 'austritt']], how='left', left_on=['nachname', 'vorname'], right_on=['nachname', 'vorname'])
del df['wohnort_x']
df = df.rename(columns={'wohnort_y': 'wohnort'})

# Einige Duplikate werden entfernt. 
df = df.drop_duplicates(subset=['vorname', 'nachname', 'geschäftsname', 'dateiname', 'register', 'datetime'])

In [154]:
'''df['wohnort'] = ''
df['wahlkreis'] = ''
df['sprache'] = ''

for i, r in tqdm(df.iterrows(), total=len(df)):
    if r['wohnort'] == '' or r['wahlkreis'] == '' or r['sprache'] == '':
        persinfos = df_grossrat[(df_grossrat['vorname'] == r['vorname']) & (df_grossrat['nachname'] == r['nachname'])]
        if len(persinfos) == 1:
            df.at[i, 'wohnort'] = persinfos['wohnort']
            df.at[i, 'sprache'] = persinfos['sprache']
            df.at[i, 'wahlkreis'] = persinfos['wahlkreis']
        else:
            print('Problem bei', i, r['nachname'], r['vorname'])'''

"df['wohnort'] = ''\ndf['wahlkreis'] = ''\ndf['sprache'] = ''\n\nfor i, r in tqdm(df.iterrows(), total=len(df)):\n    if r['wohnort'] == '' or r['wahlkreis'] == '' or r['sprache'] == '':\n        persinfos = df_grossrat[(df_grossrat['vorname'] == r['vorname']) & (df_grossrat['nachname'] == r['nachname'])]\n        if len(persinfos) == 1:\n            df.at[i, 'wohnort'] = persinfos['wohnort']\n            df.at[i, 'sprache'] = persinfos['sprache']\n            df.at[i, 'wahlkreis'] = persinfos['wahlkreis']\n        else:\n            print('Problem bei', i, r['nachname'], r['vorname'])"

In [178]:
# Die Daten werden exportiert. 
df.to_csv('daten/be_grossrat_abstimmungen.csv')